In [3]:
import psycopg2
import csv
import re

def con():
    return psycopg2.connect(
        host="localhost",
        database="lab10pgadmin",
        user="postgres",
        password="sayasaya"
    )

def insert_or_update_user(conn, name, phone):
    with conn:
        with conn.cursor() as cur:
            cur.execute("SELECT 1 FROM phonebook WHERE name = %s", (name,))
            if cur.fetchone():
                cur.execute("UPDATE phonebook SET phone = %s WHERE name = %s", (phone, name))
                print(f"UPDATED {name} with new phone {phone}")
            else:
                cur.execute("INSERT INTO phonebook (name, phone) VALUES (%s, %s)", (name, phone))
                print(f"INSERTED {name} with phone {phone}")

def insert_consol(conn):
    name = input("Enter name: ")
    phone = input("Enter phone: ")
    insert_or_update_user(conn, name, phone)

def insert_csv(path):
    invalid_data = []
    with con() as conn:
        with conn.cursor() as cur:
            with open(path, 'r') as file:
                reader = csv.reader(file)
                next(reader)  # Skip header
                for row in reader:
                    if len(row) != 2:
                        continue
                    name, phone = row
                    if re.match(r'^\d{10,15}$', phone):
                        insert_or_update_user(conn, name, phone)
                    else:
                        invalid_data.append((name, phone))
    if invalid_data:
        print("\nInvalid entries:")
        for name, phone in invalid_data:
            print(f"{name} - {phone}")

def query_pattern():
    pattern = input("Enter pattern to search (name or phone): ")
    with con() as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT * FROM phonebook 
                WHERE name ILIKE %s OR phone ILIKE %s
            """, (f'%{pattern}%', f'%{pattern}%'))
            results = cur.fetchall()
            if results:
                for row in results:
                    print(row)
            else:
                print("No matches found.")

def query_paginated():
    limit = int(input("Enter limit: "))
    offset = int(input("Enter offset: "))
    with con() as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT * FROM phonebook ORDER BY id LIMIT %s OFFSET %s", (limit, offset))
            for row in cur.fetchall():
                print(row)

def delete_user():
    target = input("Enter name or phone to delete: ")
    with con() as conn:
        with conn.cursor() as cur:
            cur.execute("DELETE FROM phonebook WHERE name = %s OR phone = %s", (target, target))
            print("Deleted if existed.")

def menu():
    while True:
        print("\n📱 PHONEBOOK MENU")
        print("1. Insert user")
        print("2. Insert from CSV (with validation)")
        print("3. Search by pattern")
        print("4. Paginated view")
        print("5. Delete by name or phone")
        print("6. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            conn = con()
            insert_consol(conn)
        elif choice == '2':
            path = input("Enter the path to the CSV file: ")
            insert_csv(path)
        elif choice == '3':
            query_pattern()
        elif choice == '4':
            query_paginated()
        elif choice == '5':
            delete_user()
        elif choice == '6':
            print("Goodbye!")
            break
        else:
            print("Invalid choice")

if __name__ == "__main__":
    menu()



📱 PHONEBOOK MENU
1. Insert user
2. Insert from CSV (with validation)
3. Search by pattern
4. Paginated view
5. Delete by name or phone
6. Exit
('nefr', '554', 2)

📱 PHONEBOOK MENU
1. Insert user
2. Insert from CSV (with validation)
3. Search by pattern
4. Paginated view
5. Delete by name or phone
6. Exit
Deleted if existed.

📱 PHONEBOOK MENU
1. Insert user
2. Insert from CSV (with validation)
3. Search by pattern
4. Paginated view
5. Delete by name or phone
6. Exit
Goodbye!
